In [19]:
import sys
sys.path.append("..")

In [20]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [21]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [22]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [23]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [24]:

# from langchain_community.document_loaders import PyPDFLoader
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        if '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版' not in filename:
            continue
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        pages += [loader.load()]
    # break # too much time, only load one file
pages

# loader = PyPDFLoader(os.path.join(dir, '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf'))
# pages += loader.load()
# pages = [pages[9]]
# pages

Processing documents: 100%|██████████| 32/32 [00:38<00:00,  1.19s/it]


[[Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='資安等級：內部\n台灣人壽\n個人保險首/續期繳費暨保單貸款還款息作業手冊\n(113 年 7 月版)_業務通路版'),
  Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 2}, page_content='\n\n1. 序號：1\n   主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明。\n   備註：無\n\n### 繳費管道變更規則\n\n#### 原指定繳費管道：金融機構\n\n1. **改自行匯款**\n   - **於一照期限內完成**：是，於一照期限內完成匯款\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n   - **未於一照期限內完成**：否，未於一照期限內完成匯款\n   - **繳費日認定原則**：以實際匯款日為繳費日\n\n2. **改他行金融機構轉帳**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n\n3. **改信用卡**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以新信用卡授權書之申請日為繳費日\n\n#### 原指定繳費管道：信用卡\n\n1. **改自行匯款**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以實際匯款日為繳費日\n\n2. **改金融機構轉帳**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以新金融機構轉帳授權書之申請日為繳費日\n\n3. **改同銀行其他信用卡或改他行信用卡**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：可以用原信用卡授權書之申請日為繳費日\n\n### 註釋\n\n- **註1**：一照意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者

In [25]:
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=5)
chunk_list = graph_builder.build_chunk_graph_with_parent_child(pages)
chunk_list

[{'chunk_id': '13ef4bab-ce5d-4420-ae60-4449c0ca8293',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'total_page_num': 28}, page_content='資安等級：內部\n台灣人壽\n個人保險首/續期繳費暨保單貸款還款息作業手冊\n(113 年 7 月版)_業務通路版\n\n\n1. 序號：1\n   主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明。\n   備註：無\n\n### 繳費管道變更規則\n\n#### 原指定繳費管道：金融機構\n\n1. **改自行匯款**\n   - **於一照期限內完成**：是，於一照期限內完成匯款\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n   - **未於一照期限內完成**：否，未於一照期限內完成匯款\n   - **繳費日認定原則**：以實際匯款日為繳費日')},
 {'chunk_id': '1c95b637-5318-4498-8b53-4a84e2f41d62',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'total_page_num': 28}, page_content='2. **改他行金融機構轉帳**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n\n3. **改信用卡**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以新信用卡授權書之申請日為繳費日\n\n#### 原指定繳費管道：信用卡\n\n1. **改自行匯款**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以實際匯款日為繳費日\n\n2. **改金融機構轉帳**\n   - **於一照期限內完成**：不限\n   - **繳費日認定原則**：以新金融機構轉帳授權書之申請日為繳費日')},
 {'ch

In [ ]:
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for chunk in chunk_list:
#     doc = chunk['chunk_doc']
#     content = doc.page_content
#     curr_num = num_tokens_from_string(content)
#     tokens_num += num_tokens_from_string(content)
# tokens_num

In [8]:
graph_builder._get_combined_chunks(chunk_list)

[Document(metadata={'combined_chunk_ids': ['c475b78a-9b67-4d6a-a70c-5e8bce455662']}, page_content='資安等級：內部 台灣人壽 個人保險首/續期繳費暨保單貸款還款息作業手冊 (113 年 7 月版)_業務通路版   1. 序號：1    主要修訂內容：配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明'),
 Document(metadata={'combined_chunk_ids': ['8b6e4073-8706-42c4-9955-5efb1135d3c2']}, page_content='。    備註：無  ### 繳費管道變更規則  #### 原指定繳費管道：金融機構  1. **改自行匯款**    - **於一照期限內完成**：是，於一照期限內完成匯款    - **繳費日認定原則**：可以用原授權書之申請日為繳費日    - **未於一照期限內完成**：否，未於一照期限內完成匯款    - **繳費日認定原則**：以實際匯款日為繳費日  2. **改他行金融機構轉帳**    - **完成**：不限    - **繳費日認定原則**：可以用原授權書之申請日為繳費日  3. **改信用卡**    - **完成**：不限    - **繳費日認定原則**：以新信用卡授權書之申請日為繳費日  #### 原指定繳費管道：信用卡  1. **改自行匯款**    - **完成**：不限    - **繳費日認定原則**：以實際匯款日為繳費日  2. **改金融機構轉帳**    - **完成**：不限    - **繳費日認定原則**：以新金融機構轉帳授權書之申請日為繳費日  3. **改同銀行其他信用卡或改他行信用卡**    - **完成**：不限    - **繳費日認定原則**：可以用原信用卡授權書之申請日為繳費日  ### 註釋  - **註1**：一照：意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者'),
 Document(metadata={'combined_chunk_ids': ['79faa07a-6722-49e3-9f4e-26853ec9d311']}, page_content='。 

In [26]:

graph_documents = graph_builder.get_graph_from_llm(llm, chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

100%|██████████| 120/120 [02:15<00:00,  1.13s/it]


[GraphDocument(nodes=[Node(id='法人', type='Concept', properties={'description': '法人是指依照法律成立的組織，具有法律人格，能夠以自己的名義享有權利和承擔義務。'}), Node(id='法人負責人', type='Concept', properties={'description': '法人負責人是指代表法人行使職權、履行義務的個人。'}), Node(id='企業員工', type='Concept', properties={'description': '企業員工是指受雇於企業，為企業工作並獲得報酬的個人。'})], relationships=[], source=Document(metadata={'combined_chunk_ids': ['89463f57-082f-430c-8ec1-84e3c328b51d']}, page_content='### 法人、法人負責人及該企業員工')),
 GraphDocument(nodes=[Node(id='保單關係人', type='Entity', properties={'description': '身分/關係證明文件：X, 繳款聲明書：X, 要/被保人：X, 受益人：V'}), Node(id='非保單關係人', type='Entity', properties={'description': '身分/關係證明文件：V【註】, 繳款聲明書：X'}), Node(id='法人、法人負責人及該企業員工', type='Entity', properties={'description': '身分/關係證明文件或法人出具之聲明書：V, 公司章程、議事錄：V, 防制洗錢及打擊資恐風險辨識問卷(法人及信託受託人適用)：V, 繳款聲明書：X'}), Node(id='其他', type='Entity', properties={'description': '無具體描述'})], relationships=[], source=Document(metadata={'combined_chunk_ids': ['484e6f5e-fa7a-44aa-9433-0d16d2095e76']}, page_content=

In [27]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [28]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='法人', type='Concept', properties={'description': '法人是指依照法律成立的組織，具有法律人格，能夠以自己的名義享有權利和承擔義務。'}), Node(id='法人負責人', type='Concept', properties={'description': '法人負責人是指代表法人行使職權、履行義務的個人。'}), Node(id='企業員工', type='Concept', properties={'description': '企業員工是指受雇於企業，為企業工作並獲得報酬的個人。'})], relationships=[], source=Document(metadata={'combined_chunk_ids': ['89463f57-082f-430c-8ec1-84e3c328b51d'], 'id': 'b9e72b91c32e5813483a6c4bbc97c9ab'}, page_content='### 法人、法人負責人及該企業員工')),
 'chunk_id': '89463f57-082f-430c-8ec1-84e3c328b51d'}

In [29]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)